In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import recall_score,accuracy_score,roc_auc_score,average_precision_score,roc_curve,auc,confusion_matrix
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.datasets import make_imbalance
from joblib import dump





In [2]:
df_base=pd.read_excel('dataset_combined.xlsx',sheet_name=0)

df_base = df_base.dropna(subset=['Diagnosis'])

mapping={
    'no':0,
    'yes':1,
    'male':1,
    'female':0,
    'normal':0,
    'diarrhea':1,
    'constipation':1,
    '+':1,
    '++':2,
    '+++':3,
    'local':1,
    'generalized':2,
    'appendicitis':1,
    'no appendicitis':0,
    'uncomplicated':0,
    'complicated':1,
}

for col in df_base.columns:
    df_base[col]=df_base[col].replace(mapping).fillna(0)

print(df_base.shape)

df=df_base[df_base['Diagnosis']==1]

print(df['Severity'].value_counts(dropna=False))

df.to_excel('dataset_complications.xlsx', index=False)



feature_active=['Nausea','Loss_of_Appetite','Peritonitis','Ketones_in_Urine','Free_Fluids','CRP','WBC_Count','Body_Temperature','Appendix_Diameter','Severity']

# feature_roster=['Peritonitis','Ketones_in_Urine','Nausea','Loss_of_Appetite','Free_Fluids','CRP','WBC_Count','Body_Temperature','Neutrophil_Percentage','Appendix_Diameter','Severity']

# print (df['Severity'].value_counts())
# print (df_base['Diagnosis'].value_counts())

(1015, 17)
Severity
0    454
1    148
Name: count, dtype: int64


In [3]:
# Model Development

df=df[feature_active]

x = df.iloc[:, 0:-1]  
y = df.iloc[:, -1]



x_temp,x_test, y_temp,y_test = train_test_split(x,y,test_size=0.2,random_state=17)


smote = SMOTE(sampling_strategy='auto', random_state=17)

# Apply SMOTE to the training data
x_temp_resampled, y_temp_resampled = smote.fit_resample(x_temp, y_temp)

# Check the distribution after resampling
print("Before SMOTE: ", y_temp.value_counts())
print("After SMOTE: ", y_temp_resampled.value_counts())


majority_class_size = y_temp_resampled.value_counts()[0]
desired_majority_size = majority_class_size // 2  # Reducing class 0 to half its size
nearmiss = NearMiss(version=1,sampling_strategy={0: desired_majority_size})

x_temp_resampled_nearmiss, y_temp_resampled_nearmiss = nearmiss.fit_resample(x_temp_resampled, y_temp_resampled)

# Distribution after NearMiss undersampling
print("After NearMiss (undersampling class 0 to half): ", y_temp_resampled_nearmiss.value_counts())

x_train,x_val,y_train,y_val = train_test_split(x_temp_resampled_nearmiss,y_temp_resampled_nearmiss,test_size=0.25,random_state=17)

print("train dataset: ", y_train.value_counts())
print("validation dataset: ", y_val.value_counts())
print("test dataset: ", y_test.value_counts())



dharma=RandomForestClassifier(
    n_estimators=555,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=33,
    random_state=17,
   
)

dharma.fit(x_train,y_train)
dump(dharma, 'DharmaComp.joblib')  # Save the model














Before SMOTE:  Severity
0    359
1    122
Name: count, dtype: int64
After SMOTE:  Severity
0    359
1    359
Name: count, dtype: int64
After NearMiss (undersampling class 0 to half):  Severity
1    359
0    179
Name: count, dtype: int64
train dataset:  Severity
1    262
0    141
Name: count, dtype: int64
validation dataset:  Severity
1    97
0    38
Name: count, dtype: int64
test dataset:  Severity
0    95
1    26
Name: count, dtype: int64


['DharmaComp.joblib']